In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.pipeline import Pipeline
import warnings
warnings.filterwarnings('ignore')

# Veri setini yükleme
url = "https://raw.githubusercontent.com/veekun/pokedex/master/pokedex/data/csv/pokemon.csv"
pokemon_df = pd.read_csv(url)

# Veri ön işleme
print("Veri seti boyutu:", pokemon_df.shape)
print("\nİlk 5 satır:")
print(pokemon_df.head())

# Eksik değerleri kontrol etme
print("\nEksik değerler:")
print(pokemon_df.isnull().sum())

# Kategorik değişkenleri sayısallaştırma
le = LabelEncoder()
pokemon_df['type1'] = le.fit_transform(pokemon_df['type1'])
pokemon_df['type2'] = le.fit_transform(pokemon_df['type2'].fillna('None'))

# Özellik seçimi
features = ['hp', 'attack', 'defense', 'sp_attack', 'sp_defense', 'speed', 'type2']
X = pokemon_df[features]
y = pokemon_df['type1']

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Modelleri tanımlama
models = {
    'Logistic Regression': Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(max_iter=1000, random_state=42))
    ]),
    'Random Forest': RandomForestClassifier(random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42),
    'SVM': Pipeline([
        ('scaler', StandardScaler()),
        ('svm', SVC(random_state=42))
    ]),
    'KNN': Pipeline([
        ('scaler', StandardScaler()),
        ('knn', KNeighborsClassifier())
    ])
}

# Modelleri eğitme ve değerlendirme
results = {}
for name, model in models.items():
    print(f"\n{name} modeli eğitiliyor...")
    # Model eğitimi
    model.fit(X_train, y_train)

    # Tahminler
    y_pred = model.predict(X_test)

    # Sonuçları kaydetme
    results[name] = {
        'accuracy': accuracy_score(y_test, y_pred),
        'classification_report': classification_report(y_test, y_pred)
    }

    print(f"\n{name} Sonuçları:")
    print(f"Doğruluk: {results[name]['accuracy']:.4f}")
    print("\nSınıflandırma Raporu:")
    print(results[name]['classification_report'])

    # Karmaşıklık matrisi
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'{name} - Karmaşıklık Matrisi')
    plt.xlabel('Tahmin Edilen')
    plt.ylabel('Gerçek')
    plt.show()

# En iyi modeli bulma
best_model_name = max(results.items(), key=lambda x: x[1]['accuracy'])[0]
print(f"\nEn iyi model: {best_model_name} (Doğruluk: {results[best_model_name]['accuracy']:.4f})")

# En iyi model için hiper parametre optimizasyonu
print(f"\n{best_model_name} için hiper parametre optimizasyonu yapılıyor...")

# Hiper parametre dağılımlarını tanımlama
param_distributions = {
    'Logistic Regression': {
        'lr__C': [0.1, 1.0, 10.0, 100.0],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear', 'saga']
    },
    'Random Forest': {
        'n_estimators': [100, 200, 300],
        'max_depth': [10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4]
    },
    'Gradient Boosting': {
        'n_estimators': [100, 200, 300],
        'max_depth': [3, 5, 7],
        'learning_rate': [0.01, 0.1, 0.2],
        'subsample': [0.8, 0.9, 1.0]
    },
    'SVM': {
        'svm__C': [0.1, 1.0, 10.0],
        'svm__kernel': ['rbf', 'linear'],
        'svm__gamma': ['scale', 'auto']
    },
    'KNN': {
        'knn__n_neighbors': [3, 5, 7, 9],
        'knn__weights': ['uniform', 'distance'],
        'knn__metric': ['euclidean', 'manhattan']
    }
}

# RandomizedSearchCV için model ve parametre dağılımını seçme
model = models[best_model_name]
param_dist = param_distributions[best_model_name]

# RandomizedSearchCV ile hiper parametre optimizasyonu
random_search = RandomizedSearchCV(
    estimator=model,
    param_distributions=param_dist,
    n_iter=20,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# RandomizedSearchCV'yi çalıştırma
random_search.fit(X_train, y_train)

# En iyi parametreleri ve skoru yazdırma
print("\nEn iyi parametreler:")
print(random_search.best_params_)
print(f"\nEn iyi cross-validation skoru: {random_search.best_score_:.4f}")

# En iyi model ile test seti üzerinde tahmin
best_model = random_search.best_estimator_
y_pred = best_model.predict(X_test)

# Test seti sonuçlarını yazdırma
print("\nTest seti sonuçları:")
print(f"Doğruluk: {accuracy_score(y_test, y_pred):.4f}")
print("\nSınıflandırma Raporu:")
print(classification_report(y_test, y_pred))

# Karmaşıklık matrisi
cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title(f'Optimize Edilmiş {best_model_name} - Karmaşıklık Matrisi')
plt.xlabel('Tahmin Edilen')
plt.ylabel('Gerçek')
plt.show()

# Tüm modellerin doğruluk skorlarını karşılaştırma
accuracies = {name: result['accuracy'] for name, result in results.items()}
plt.figure(figsize=(10, 6))
plt.bar(accuracies.keys(), accuracies.values())
plt.title('Model Doğruluk Skoru Karşılaştırması')
plt.xticks(rotation=45)
plt.ylabel('Doğruluk Skoru')
plt.show()